<h1>Analysis - Average Fare<h1>

Setting up environment

In [4]:
from pyspark.sql import SparkSession
import pandas as pd
import geopandas as gpd
import folium
from pyspark.sql import functions as F
#import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from pingouin import ancova

In [ ]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .getOrCreate()
)

Read in the data

In [ ]:
df = spark.read.parquet('../data/curated/combined_data')

<h3>Analysis of location vs average driver pay<h3>

In [ ]:
# Getting geodata

zones = pd.read_csv("../data/curated/taxi_zones/taxi+_zone_lookup.csv")
sf = gpd.read_file("../data/curated/taxi_zones/taxi_zones.shp")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
gdf = gpd.GeoDataFrame(
    pd.merge(zones, sf, on='LocationID', how='inner')
)

geoJSON = gdf[['LocationID', 'geometry']].drop_duplicates('LocationID').to_json()

In [ ]:
# Aggregating data over location

location_avgs = df.groupby('PU_Location_ID').avg('Driver_pay').withColumnRenamed('avg(Driver_pay)', 'Avg_Driver_Pay').toPandas()
location_avgs = location_avgs \
    .merge(gdf[['LocationID', 'geometry']], left_on='PU_Location_ID', right_on='LocationID') \
    .drop('LocationID', axis=1)

In [ ]:
m = folium.Map(location=[40.73, -73.74], tiles="Stamen Terrain", zoom_start=10)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=location_avgs, # data source
    columns=['PU_Location_ID','Avg_Driver_Pay'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Average Trip Earnings USD$'
)

c.add_to(m)

m

Clear correlation, airports particularly have larger prices (as expected). Also larger prices in areas around central park and broadway, busy areas.

In [24]:
pdf = pd.read_parquet("../data/curated/combined_data_sample")
pdf = pdf.rename(columns={'Humidity_%': 'Humidity'})

In [16]:
fit = ols(
    formula="Driver_pay ~ C(PU_Location_ID)",
    data=pdf
).fit()

In [17]:
table = sm.stats.anova_lm(fit, typ=2)
print(table)

                         sum_sq        df          F  PR(>F)
C(PU_Location_ID)  1.935288e+06     257.0  57.978172     0.0
Residual           1.404359e+07  108126.0        NaN     NaN


<h3>Analysis of time (day of week and hour) vs average driver pay<h3>

Sub-sample one location for visualisation

In [ ]:
time_visualisation_df = df.groupBy("PU_Location_ID", "Day_of_week").avg("Driver_pay").toPandas()

In [ ]:
time_visualisation_df.boxplot('avg(Driver_pay)', 'Day_of_week')

In [ ]:
time_visualisation_df = df.groupBy("PU_Location_ID", "Hour").avg("Driver_pay").toPandas()
time_visualisation_df.boxplot('avg(Driver_pay)', 'Hour')

In [18]:
fit = ols(
    formula="Driver_pay ~ C(Day_of_week) + C(Hour)",
    data=pdf
).fit()

In [19]:
table = sm.stats.anova_lm(fit, typ=2)
print(table)

                      sum_sq        df          F         PR(>F)
C(Day_of_week)  1.078760e+04       6.0  12.282843   7.196653e-14
C(Hour)         1.047404e+05      23.0  31.110828  7.112811e-136
Residual        1.586060e+07  108354.0        NaN            NaN


<h3>Analysis of weather vs average driver pay<h3>

In [25]:
fit = ols(
    formula="Driver_pay ~ Temperature_C + Speed_kmh + Precip_Rate_mm + Humidity + Pressure_hPa",
    data=pdf
).fit()

In [26]:
table = sm.stats.anova_lm(fit, typ=2)
print(table)

                      sum_sq        df          F        PR(>F)
Temperature_C   1.156987e+04       1.0  78.556481  7.888584e-19
Speed_kmh       5.039288e+00       1.0   0.034215  8.532496e-01
Precip_Rate_mm  2.227802e+03       1.0  15.126208  1.006175e-04
Humidity        8.828258e+02       1.0   5.994162  1.435487e-02
Pressure_hPa    1.517085e+02       1.0   1.030062  3.101464e-01
Residual        1.596201e+07  108378.0        NaN           NaN


In [20]:
ancova(data=pdf, dv='Driver_pay', covar=['Temperature_C', 'Humidity_%', 'Speed_kmh', 'Precip_Rate_mm', 'Pressure_hPa'], between='PU_Location_ID')

,Source,SS,DF,F,p-unc,np2
0,PU_Location_ID,1.936975e+06,257,58.102765,0.000000e+00,0.121349
1,Temperature_C,1.140361e+04,1,87.912058,6.969159e-21,0.000812
2,Humidity_%,2.554035e+03,1,19.689411,9.119464e-06,0.000182
3,Speed_kmh,4.824573e+02,1,3.719331,5.378852e-02,0.000034
4,Precip_Rate_mm,2.118788e+03,1,16.334035,5.313831e-05,0.000151
5,Pressure_hPa,2.703826e+02,1,2.084418,1.488119e-01,0.000019
6,Residual,1.402504e+07,108121,NaN,NaN,NaN


Pressure unecessary